### Imports

In [1]:
import numpy as np
import tensorflow as tf

### Load the data

In [2]:
# Load the training data first:
npz = np.load('financial_data_train.npz')

train_inputs = npz['inputs'].astype(np.float64)
train_targets = npz['targets'].astype(np.int32)

# Load the validation data next:
npz = np.load('financial_data_validation.npz')

validation_inputs = npz['inputs'].astype(np.float64)
validation_targets = npz['targets'].astype(np.int32)

# Finally, load the test data:
npz = np.load('financial_data_test.npz')

test_inputs = npz['inputs'].astype(np.float64)
test_targets = npz['targets'].astype(np.int32)

###Neural Network model

Model architecture

In [14]:
input_size = 47 # Number of features in the input data set.
output_size = 2 # Since the outcome is binary.

hidden_layer_size = int(2/3 * input_size + output_size)
# (input_size + output_size) // 2

dropout_rate = 0.2

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_size,)),  # Input layer
    tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'), # Hidden Layer 1
    tf.keras.layers.Dropout(dropout_rate), # Dropout for H/ L 1
    tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'), # Hidden Layer 2
    tf.keras.layers.Dropout(dropout_rate), # Dropout for H/ L 2
    # tf.keras.layers.Dense(hidden_layer_size, activation= 'sigmoid'), # Hidden Layer 3
    # tf.keras.layers.Dense(dropout_rate) # Dropout for H/ L 3
    tf.keras.layers.Dense(output_size, activation= 'softmax') # Output layer
])

Model hyperparameters and training:

In [15]:
optimizer = tf.keras.optimizers.SGD(learning_rate= 0.01, momentum= 0.9)

model.compile(
    optimizer= optimizer,
    loss= 'sparse_categorical_crossentropy', # Since the outcome is binary (0 or 1) 
    metrics= ['accuracy'] # parameters over which the model accuracy is evaluated
)

batch_size = 32
max_epochs = 100 # Training will stop after 100 cycles if callback doesn't kick in.

# Implementing callback -> EarlyStopping with a patience of 10 so that
# if the validation accuracy doesn't increase for 10 consecutive trainin
# epochs we can stop the training of model.
# This helps to prevent overfitting.
early_stopping = tf.keras.callbacks.EarlyStopping(monitor= 'val_accuracy', patience= 15, restore_best_weights= True)
# If the validation accuracy hit a plateau (doesn;t show substantial change),
# learning rate will be reduced by a factor of 0.2 untill it hits a minimum of 0.001.
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor= 'val_accuracy', factor= 0.2, patience= 2, min_lr= 0.001)

# Give the model data to train on.
history = model.fit(
    train_inputs,
    train_targets,
    batch_size = batch_size,
    epochs= max_epochs,
    callbacks= [early_stopping, reduce_lr],
    validation_data= (validation_inputs, validation_targets),
    verbose= 2
)

Epoch 1/100
2605/2605 - 3s - 1ms/step - accuracy: 0.5241 - loss: 0.6917 - val_accuracy: 0.5416 - val_loss: 0.6863 - learning_rate: 0.0100
Epoch 2/100
2605/2605 - 2s - 940us/step - accuracy: 0.5431 - loss: 0.6863 - val_accuracy: 0.5519 - val_loss: 0.6818 - learning_rate: 0.0100
Epoch 3/100
2605/2605 - 3s - 1ms/step - accuracy: 0.5572 - loss: 0.6816 - val_accuracy: 0.5411 - val_loss: 0.6827 - learning_rate: 0.0100
Epoch 4/100
2605/2605 - 2s - 885us/step - accuracy: 0.5648 - loss: 0.6777 - val_accuracy: 0.5752 - val_loss: 0.6707 - learning_rate: 0.0100
Epoch 5/100
2605/2605 - 3s - 1ms/step - accuracy: 0.5725 - loss: 0.6726 - val_accuracy: 0.5757 - val_loss: 0.6700 - learning_rate: 0.0100
Epoch 6/100
2605/2605 - 2s - 870us/step - accuracy: 0.5765 - loss: 0.6683 - val_accuracy: 0.5878 - val_loss: 0.6637 - learning_rate: 0.0100
Epoch 7/100
2605/2605 - 2s - 842us/step - accuracy: 0.5815 - loss: 0.6657 - val_accuracy: 0.5982 - val_loss: 0.6607 - learning_rate: 0.0100
Epoch 8/100
2605/2605 - 2s